In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from uncertainties import unumpy, ufloat

Counting a set number of fringes, then measuring the change in distance of the back mirror (micrometers). We took the average of our counts for each measurement

In [8]:
num_fringes_10 = unumpy.uarray([3.4, 3.5], [0.1, 0.1])
num_fringes_26 = unumpy.uarray([8.9, 8.4], [0.1, 0.1])
num_fringes_50 = unumpy.uarray([17.0], [0.1])
num_fringes_55 = unumpy.uarray([17.5], [0.1])
num_fringes_78 = unumpy.uarray([25.5], [0.1])
num_fringes_75 = unumpy.uarray([24.7], [0.1])
num_fringes_102 = unumpy.uarray([33.1], [0.1])
num_fringes_101 = unumpy.uarray([33.7], [0.1])


10 degrees:  [3.4+/-0.1 3.5+/-0.1]


In [12]:
## FIXME: doesn't make sense to call the measured distane "num_fringes" - should be "distance"

def wavelength(num_fringes, distance):
    return (2*num_fringes*10**-6)/(distance)

wavelength_10_01 = wavelength(num_fringes_10[0], 10)
wavelength_10_02 = wavelength(num_fringes_10[1], 10)
wavelength_26_01 = wavelength(num_fringes_26[0], 26)
wavelength_26_02 = wavelength(num_fringes_26[1], 26)
wavelength_50 = wavelength(num_fringes_50, 50)
wavelength_55 = wavelength(num_fringes_55, 55)
wavelength_78 = wavelength(num_fringes_78, 78)
wavelength_75= wavelength(num_fringes_75, 75)
wavelength_102 = wavelength(num_fringes_102, 102)
wavelength_101 = wavelength(num_fringes_101, 101)

wavelength_avg = (wavelength_10_01 + wavelength_10_02 + wavelength_26_01 + wavelength_26_02 + wavelength_50 + wavelength_55 + wavelength_78 + wavelength_75 + wavelength_102 + wavelength_101)/10
print("Average wavelength:", wavelength_avg)

Average wavelength: [6.655992028162091e-07+/-3.1129052798873734e-09]


### Measured the zeroth order and the second order from both the right and left sides. This was done to determine if the diffraction grating was perpendicular to the incident beam. We took the differences from both second order measurements and the zeroth order, then averaged them out to determine what the angle for both the second order measurements should be for the grating to be considered perpendicular to the incident beam.

In [13]:
# Measurements for diffraction grating

zeroth_order_will = unumpy.uarray([118 + 53/60], [1/60])
zeroth_order_matthew = unumpy.uarray([118 + 53/60], [1/60])
second_order_right_side_will = unumpy.uarray([172 + 18/60], [1/60])
second_order_right_side_matthew = unumpy.uarray([172 + 20/60], [1/60])
second_order_left_side_will = unumpy.uarray([168 + 30/60], [1/60])
second_order_left_side_matthew = unumpy.uarray([], [1/60])

In [14]:
def diffraction_grating_calc(order, wavelength, angle):
    return (order*wavelength)/(unumpy.sin(angle*np.pi/180))

diffraction_grating_will = diffraction_grating_calc(2, wavelength_avg, second_order_right_side_will - zeroth_order_will)
print("Diffraction grating constant (Will):", diffraction_grating_will)
diffraction_grating_matthew = diffraction_grating_calc(2, wavelength_avg, second_order_right_side_matthew - zeroth_order_matthew)
print("Diffraction grating constant (Matthew):", diffraction_grating_matthew)

Diffraction grating constant (Will): [1.6578001636193154e-06+/-7.76978230861035e-09]
Diffraction grating constant (Matthew): [1.6570849102160255e-06+/-7.766390028982182e-09]


In [25]:
# Try measuring the diffraction grating now that it is perpendicular to the incident light within 1 degree of error

second_order_left_side_will = unumpy.uarray([52 + 33/60], [1/60])
diffraction_grating_second_order_left_side_will = diffraction_grating_calc(2, wavelength_avg, second_order_left_side_will)

first_order_left_side_matthew = unumpy.uarray([23 + 33/60], [1/60])
diffraction_grating_matthew_first_order_left_side_matthew = diffraction_grating_calc(1, wavelength_avg, first_order_left_side_matthew)

first_order_right_side_matthew = unumpy.uarray([23 + 4/60], [1/60])
diffraction_grating_first_order_right_side_matthew = diffraction_grating_calc(1, wavelength_avg, first_order_right_side_matthew)

second_order_right_side_matthew = unumpy.uarray([51 + 38/60], [1/60])
diffraction_grating_second_order_right_side_matthew = diffraction_grating_calc(2, wavelength_avg, second_order_right_side_matthew)

# Average the diffraction grating constants
diffraction_grating_avg = (diffraction_grating_will + diffraction_grating_matthew + diffraction_grating_second_order_left_side_will + diffraction_grating_matthew_first_order_left_side_matthew + diffraction_grating_first_order_right_side_matthew + diffraction_grating_second_order_right_side_matthew)/6
print("Average diffraction grating constant:", diffraction_grating_avg)

# Calculate slits per mm
slits_per_mm = 1/diffraction_grating_avg*10**-3``
print("Slits per mm:", slits_per_mm)

Average diffraction grating constant: [1.6856669624415015e-06+/-7.889184088292978e-09]
Slits per mm: [593.236993001044+/-2.776441580721214]


# Helium

In [46]:
def wavelength_calc(order, angle):
    return (diffraction_grating_avg)*unumpy.sin(angle*np.pi/180)/order


# Zeroth order
zeroth_order = unumpy.uarray([134 + 54/60], [1/60])


# First order left side
first_order_left_violet = unumpy.uarray([119 + 19/60], [1/60])
first_order_left_violet_wavelength = wavelength_calc(1, zeroth_order - first_order_left_violet)
print("First order left violet wavelength:", first_order_left_violet_wavelength)

first_order_left_blue = unumpy.uarray([118 + 29/60], [1/60])
first_order_left_blue_wavelength = wavelength_calc(1, zeroth_order - first_order_left_blue)
print("First order left blue wavelength:", first_order_left_blue_wavelength)

first_order_left_cyan = unumpy.uarray([117 + 44/60], [1/60])
first_order_left_cyan_wavelength = wavelength_calc(1, zeroth_order - first_order_left_cyan)
print("First order left cyan wavelength:", first_order_left_cyan_wavelength)

first_order_left_green = unumpy.uarray([117 + 22/60], [1/60])
first_order_left_green_wavelength = wavelength_calc(1, zeroth_order - first_order_left_green)
print("First order left green wavelength:", first_order_left_green_wavelength)

first_order_left_orange = unumpy.uarray([114 + 14/60], [1/60])
first_order_left_orange_wavelength = wavelength_calc(1, zeroth_order - first_order_left_orange)
print("First order left orange wavelength:", first_order_left_orange_wavelength)

first_order_left_red = unumpy.uarray([111 + 16/60], [1/60])
first_order_left_red_wavelength = wavelength_calc(1, zeroth_order - first_order_left_red)
print(r"First order left red wavelength #1:", first_order_left_red_wavelength)

first_order_left_red_2 = unumpy.uarray([109 + 50/60], [1/60])
first_order_left_red_wavelength_2 = wavelength_calc(1, zeroth_order - first_order_left_red_2)
print(r"First order left red wavelength #2:", first_order_left_red_wavelength_2)


# Second order left side
second_order_left_violet = unumpy.uarray([102 + 30/60], [1/60])
second_order_left_violet_wavelength = wavelength_calc(2, zeroth_order - second_order_left_violet)
print("\nSecond order left violet wavelength:", second_order_left_violet_wavelength)

second_order_left_blue = unumpy.uarray([100 + 39/60], [1/60])
second_order_left_blue_wavelength = wavelength_calc(2, zeroth_order - second_order_left_blue)
print("Second order left blue wavelength:", second_order_left_blue_wavelength)

second_order_left_cyan = unumpy.uarray([98 + 44/60], [1/60])
second_order_left_cyan_wavelength = wavelength_calc(2, zeroth_order - second_order_left_cyan)
print("Second order left cyan wavelength:", second_order_left_cyan_wavelength)

second_order_left_green = unumpy.uarray([97 + 59/60], [1/60])
second_order_left_green_wavelength = wavelength_calc(2, zeroth_order - second_order_left_green)
print("Second order left green wavelength:", second_order_left_green_wavelength)

second_order_left_orange = unumpy.uarray([90 + 13/60], [1/60])
second_order_left_orange_wavelength = wavelength_calc(2, zeroth_order - second_order_left_orange)
print("Second order left orange wavelength:", second_order_left_orange_wavelength)

second_order_left_red = unumpy.uarray([81 + 59/60], [1/60])
second_order_left_red_wavelength = wavelength_calc(2, zeroth_order - second_order_left_red)
print(r"Second order left red wavelength #1:", second_order_left_red_wavelength)

First order left violet wavelength: [4.5283696023248784e-07+/-2.2221158836553864e-09]
First order left blue wavelength: [4.7640403641643865e-07+/-2.3267523109699245e-09]
First order left cyan wavelength: [4.975283825710015e-07+/-2.4209373587042773e-09]
First order left green wavelength: [5.07825038170177e-07+/-2.4669677187170656e-09]
First order left orange wavelength: [5.949233929739913e-07+/-2.858931252680043e-09]
First order left red wavelength #1: [6.757536831491414e-07+/-3.2258074915788932e-09]
First order left red wavelength #2: [7.141707570733979e-07+/-3.400940185682319e-09]

Second order left violet wavelength: [4.5161276294350525e-07+/-2.1337954355915647e-09]
Second order left blue wavelength: [4.743508364574275e-07+/-2.2384587225087886e-09]
Second order left cyan wavelength: [4.973864612120759e-07+/-2.344614649206736e-09]
Second order left green wavelength: [5.062503062387308e-07+/-2.385491741580936e-09]
Second order left orange wavelength: [5.926703140268443e-07+/-2.78472273